# Market trades - summary statistics

In [5]:
from aspidoceleon.bigquery import BigQuery
from aspidoceleon.time_utils import convert_epoch_seconds_to_timestamp
from charadrius.const import Const
from loguru import logger
import pandas as pd
import datetime
import os
constants = Const()
bigquery = BigQuery()

# workdir
workdir = f"{constants.PROJECT_WORKDIR}/market_trades"
os.makedirs(workdir, exist_ok=True)
logger.info("workdir")

# This week, Sunday to today inclusive
TODAY = datetime.datetime.now().strftime(constants.DATE_FMT_NODASH)
WEEK_START = pd.to_datetime(datetime.datetime.now()).to_period("W-SAT").start_time.strftime(constants.DATE_FMT_NODASH)
logger.info(f"Date range: {WEEK_START}, {TODAY}")

2024-06-25 20:16:36.926 | INFO     | __main__:<module>:14 - workdir
2024-06-25 20:16:36.928 | INFO     | __main__:<module>:19 - Date range: 20240623, 20240625


In [6]:
"""
Fetch market trades RT data
"""
fp_rt = f"{workdir}/market_trades_{WEEK_START}_{TODAY}.parquet"
sql_rt = f"""
select
    *
from `fastitocalon.coinbase.rt_coinbase_market_trades_*`
where _table_suffix between '{WEEK_START}' and '{TODAY}'
"""
df_rt = bigquery.query_cache(sql=sql_rt, fp=fp_rt)
df_rt = convert_epoch_seconds_to_timestamp(df_rt)
logger.info(f"fetched shape {df_rt.shape}")

2024-06-25 20:16:40.201 | INFO     | aspidoceleon.bigquery:query_cache:131 - fetching -> /mnt/vol1/charadrius/market_trades/market_trades_20240623_20240625.parquet
2024-06-25 20:16:41.122 | INFO     | aspidoceleon.bigquery:estimate_query_cost:89 - Estimated query cost: $0.00 (0.1 GB)


In [ ]:
df_rt.head()

## Compare against TARDIS.DEV data (silver source)